In [4]:
from dsvr.datasets.robot_data import RoboticsDatasetV2
from dsvr.results.robot_results import VisionInferenceResultV3, ADDResultV2

In [3]:
icp_results_path = "/home/aditya/Documents/workspace/inverse_graphics/output/October2025/21/16/icp/results/vision_results.npz"
fp_results_path = "/home/aditya/Documents/working/oct2025/vision_inference_results.npz"
bc_pf_results_path = "/home/aditya/Documents/workspace/inverse_graphics/output/November2025/04/10/sir/results/vis_results.npz"
dataset_path = "home/aditya/Documents/workspace/inverse_graphics/output/October2025/20/14/noisy_image_data/data/dataset.npz"

ds = RoboticsDatasetV2.load(dataset_path)
icp_res = VisionInferenceResultV3.load(icp_results_path)
fp_res = VisionInferenceResultV3.load(fp_results_path)
bc_pf_res = VisionInferenceResultV3.load(bc_pf_results_path)


In [ ]:
fp_add = 